# Habit Moment Analysis

Data file `companyclean.csv` comes from Company's online transaction data from September 2017 to December 2018.

In [2]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# import itertools
# import datetime as dt

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# 1. Reading and preparing the data for analysis

In [3]:
df = pd.read_csv('companyclean.csv')

In [8]:
df.head()

,order_date;registered_date;user_id;SKU;Quantity;;
0,4 Sep 2017;4 Sep 2017;80879;50917;1;;
1,4 Sep 2017;4 Sep 2017;80880;50917;1;;
2,4 Sep 2017;4 Sep 2017;80880;50917;1;;
3,4 Sep 2017;29 Aug 2017;80875;50917;1;;
4,4 Sep 2017;29 Aug 2017;80875;50917;1;;


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8458 entries, 0 to 8457
Data columns (total 1 columns):
order_date;registered_date;user_id;SKU;Quantity;;    8458 non-null object
dtypes: object(1)
memory usage: 66.2+ KB


In [10]:
df = df['order_date;registered_date;user_id;SKU;Quantity;;'].str.split(';', expand=True)

In [11]:
df.head()

,0,1,2,3,4,5,6
0,4 Sep 2017,4 Sep 2017,80879,50917,1,,
1,4 Sep 2017,4 Sep 2017,80880,50917,1,,
2,4 Sep 2017,4 Sep 2017,80880,50917,1,,
3,4 Sep 2017,29 Aug 2017,80875,50917,1,,
4,4 Sep 2017,29 Aug 2017,80875,50917,1,,


In [12]:
df['order_date'] = pd.to_datetime(df[0])
df['register_date'] = pd.to_datetime(df[1])
df['user_id'] = df[2]
df['quantity'] = pd.to_numeric(df[4])

In [13]:
df['cohortGroup'] = df['register_date'].apply(lambda x: x.strftime('%Y-%m'))

# 2. Find no. of users who conducted or did not conduct an action within a certain time period

In [14]:
def activation_behaviour(no_of_days, no_of_orders):
    
    '''
    Example of use: activation_behaviour(30, 1) >> How many people made 1 order within 30 days of their registration?
    '''
    
    filterByDaysAndSumQuantity = df[df['order_date'] <= (df['register_date'] + pd.DateOffset(days=no_of_days))].groupby(['cohortGroup','user_id'])['quantity'].sum().reset_index() 
    OrderNo = filterByDaysAndSumQuantity['quantity'] == no_of_orders
    filterByDaysAndOrders = filterByDaysAndSumQuantity[OrderNo]    
    activation_list_cohort = filterByDaysAndOrders.groupby(['cohortGroup'])['user_id'].nunique().to_frame('No. of Users').reset_index()
    
    return activation_list_cohort

In [15]:
activation_behaviour(30,1)        #change parameters accordingly based on no. of days and no. of orders

,cohortGroup,No. of Users
0,2017-09,145
1,2017-10,153
2,2017-11,161
3,2017-12,78
4,2018-01,158
5,2018-02,236
6,2018-03,174
7,2018-04,317
8,2018-05,541
9,2018-06,217


In [16]:
def activation_behaviour_negative(no_of_days, no_of_orders):
    
    '''
    Example of use: activation_behaviour(30, 1) >> How many people did not make 1 order within 30 days of their registration?
    '''
    
    filterByDaysAndSumQuantity = df[df['order_date'] <= (df['register_date'] + pd.DateOffset(days=no_of_days))].groupby(['cohortGroup','user_id'])['quantity'].sum().reset_index() 
    OrderNo = filterByDaysAndSumQuantity['quantity'] != no_of_orders
    filterByDaysAndOrders = filterByDaysAndSumQuantity[OrderNo]    
    activation_list_cohort = filterByDaysAndOrders.groupby(['cohortGroup'])['user_id'].nunique().to_frame('No. of Users').reset_index()
    
    return activation_list_cohort

In [17]:
activation_behaviour_negative(30,3)          #change parameters accordingly based on no. of days and no. of orders

,cohortGroup,No. of Users
0,2017-08,1
1,2017-09,161
2,2017-10,179
3,2017-11,182
4,2017-12,88
5,2018-01,189
6,2018-02,264
7,2018-03,211
8,2018-04,392
9,2018-05,653


# 3. Find no. of users who completed long term retention

Long term retention: users who conducted a certain action within the first 30 days of their registered date, who came back and did the same action in the last 3 months (October 2018 to December 2018).

In [18]:
def longtermretention_completed(no_of_orders):
    LTlast3months = df['order_date'] >= '01-10-2018'
    LTlast3monthsTable = df[LTlast3months]
    LTlast3monthsTableClean = LTlast3monthsTable.groupby(['cohortGroup','user_id'])['quantity'].sum().reset_index()
    LTl3tc1 = LTlast3monthsTableClean['quantity'] == no_of_orders
    LTl3tc2 = LTlast3monthsTableClean[LTl3tc1]                #list of people who purchased #orders in last 3 months
    LT1a = df[df['order_date'] <= (df['register_date'] + pd.DateOffset(days=30))].groupby(['cohortGroup','user_id'])['quantity'].sum().reset_index() 
    LT1b = LT1a['quantity'] == no_of_orders 
    LT1c = LT1a[LT1b]                                         #list of people who purchased #orders in first 30 days  
    booleanlist = LT1c['user_id'].isin(LTl3tc2['user_id'])    #From list of people who purchased #orders in 30 days, how many purchase #orders in last 3 months
    LT1cBoolean = LT1c[booleanlist]                     
    Result = LT1cBoolean.groupby(['cohortGroup'])['user_id'].nunique().to_frame('No. of Users').reset_index()
    return Result

In [25]:
longtermretention_completed(1)       #change parameter accordingly based on no. of orders


,cohortGroup,No. of Users
0,2017-09,16
1,2017-10,16
2,2017-11,19
3,2017-12,12
4,2018-01,108
5,2018-02,200
6,2018-03,127
7,2018-04,269
8,2018-05,474
9,2018-06,192


# 4. Find no. of users who did not complete long term retention

Users who did not complete long term retention are automatically computed based on the following formula:

             Total no. of users in a cohort - No. of users who completed long term retention in the cohort